#### 2.1 调用Chat模式

- 调用并打印回答内容：

In [1]:
from bida import ChatLLM

llm = ChatLLM(model_type='baidu')
result = llm.chat("你好呀，请问你是谁？") 
print(result)


你好，我是文心一言，英文名是ERNIE Bot。我可以完成的任务包括知识问答，文本创作，知识推理，数学计算，代码理解与编写，作画，翻译等。


- 流式stream调用，实现打字机效果：

In [2]:
from bida import ChatLLM

# stream调用
llm = ChatLLM(model_type="baidu", stream_callback=ChatLLM.stream_callback_func)

result = llm.chat("你好呀，请问你是谁？")      

你好，我是文心一言，英文名是ERNIE Bot。我可以完成的任务包括知识问答，文本创作，知识推理，数学计算，代码理解与编写，作画，翻译等。

- 迭代方式调用


In [4]:
from bida import ChatLLM

llm = ChatLLM(model_type='baidu')
def callllm():
    for partial_message in llm.achat("你好，请问你的名字是？"):     # 默认为返回全量
        yield partial_message
for content in callllm():
    print(content)

你好，我没有特定的名字
你好，我没有特定的名字。
你好，我没有特定的名字。如果你有需要，可以随时给我起一个你喜欢的名字。


In [6]:
from bida import ChatLLM

llm = ChatLLM(model_type='baidu')
def callllm():
    for partial_message in llm.achat("你好，请问你的名字是？", increment=True):     # 改为增量返回
        yield partial_message
for content in callllm():
    print(content, end="", flush=True)

你好，我没有特定的名字。如果你有需要，可以随时给我起一个你喜欢的名字。

- 更换调用的模型，使用非默认模型

In [3]:

from bida import ChatLLM

llm = ChatLLM(
    model_type="baidu", 
    stream_callback=ChatLLM.stream_callback_func, 
    model_name="ernie-bot-turbo"                # 更换到turbo模型
    ) 

result = llm.chat("你是谁？")

您好，我是百度研发的知识增强大语言模型，中文名是文心一言，英文名是ERNIE Bot。我能够与人对话互动，回答问题，协助创作，高效便捷地帮助人们获取信息、知识和灵感。

- 多轮对话：

In [2]:
from bida import ChatLLM

llm = ChatLLM(
    model_type="baidu",
    stream_callback=ChatLLM.stream_callback_func) 

result = llm.chat("你是一个服务助理，请简洁回答我的问题。我的名字是老周，年龄30岁。")

好的，老周。请问有什么我可以帮助您的吗？

In [3]:
result = llm.chat("我的名字是？")

您告诉我您的名字是老周。如果您有任何需要帮助的地方，请随时告诉我。

In [4]:
result = llm.chat("我多大了？")

您告诉我您今年30岁。如果您有其他任何问题或者需要帮助，请随时告诉我，我将竭诚为您服务。

In [5]:
# 打印聊天历史
llm.conversation.history_messages

[{'role': 'user', 'content': '你是一个服务助理，请简洁回答我的问题。我的名字是老周，年龄30岁。'},
 {'role': 'assistant', 'content': '好的，老周。请问有什么我可以帮助您的吗？'},
 {'role': 'user', 'content': '我的名字是？'},
 {'role': 'assistant', 'content': '您告诉我您的名字是老周。如果您有任何需要帮助的地方，请随时告诉我。'},
 {'role': 'user', 'content': '我多大了？'},
 {'role': 'assistant',
  'content': '您告诉我您今年30岁。如果您有其他任何问题或者需要帮助，请随时告诉我，我将竭诚为您服务。'}]

- 会话持久化

In [17]:
from bida import ChatLLM

llm = ChatLLM(
    model_type="baidu",
    auto_save_conversation=True,    # 设为自动保存会话 
    stream_callback=ChatLLM.stream_callback_func) 

result = llm.chat("你是一个服务助理，请简洁回答我的问题。我的名字是老周，年龄30岁。")


好的，老周。请问有什么我可以帮助您的吗？

In [18]:
import duckdb
conn = duckdb.connect('db/bida.db')

sql_select_conversation = """
SELECT conversation_id, conversation_title, conversation_model_type, persist_time
FROM conversation
WHERE conversation_id = ?
"""
# 根据当前会话的ID查找数据库中的会话数据
result = conn.execute(sql_select_conversation, (llm.conversation.conversation_id, )).fetchall()
print(len(result))
result

1


[('7d920c2b-4916-4772-8650-d400269e3a48',
  '你是一个服务助理，请简洁回答我的问题。我',
  'baidu',
  datetime.datetime(2023, 9, 9, 18, 28, 33, 260562))]

In [19]:
sql_select_message = """
SELECT *
FROM conversation_message
WHERE conversation_id = ?
ORDER BY end_time ASC
"""
# 根据当前会话的ID查找数据库中的消息数据
# 消息数据在数据库中是多版本存储，只要修改过就会保存一个版本，详细请看源代码
result = conn.execute(sql_select_message, (llm.conversation.conversation_id,)).fetchall()
print(len(result))
result

4


[('d120324b-dc82-4e53-ad69-5c6b046134da',
  'user',
  '你是一个服务助理，请简洁回答我的问题。我的名字是老周，年龄30岁。',
  '你是一个服务助理，请简洁回答我的问题。我的名字是老周，年龄30岁。',
  None,
  None,
  0,
  datetime.datetime(2023, 9, 9, 18, 28, 31, 594385),
  datetime.datetime(2023, 9, 9, 18, 28, 31, 594385),
  'llm',
  'processing',
  None,
  datetime.datetime(2023, 9, 9, 18, 28, 31, 594385),
  '7d920c2b-4916-4772-8650-d400269e3a48'),
 ('d120324b-dc82-4e53-ad69-5c6b046134da',
  'user',
  '你是一个服务助理，请简洁回答我的问题。我的名字是老周，年龄30岁。',
  '你是一个服务助理，请简洁回答我的问题。我的名字是老周，年龄30岁。',
  None,
  None,
  28,
  datetime.datetime(2023, 9, 9, 18, 28, 31, 594385),
  datetime.datetime(2023, 9, 9, 18, 28, 31, 594385),
  'llm',
  'completed',
  None,
  datetime.datetime(2023, 9, 9, 18, 28, 33, 259535),
  '7d920c2b-4916-4772-8650-d400269e3a48'),
 ('2dad9504-7959-4a74-b71c-f945bfe1d44a',
  'assistant',
  None,
  None,
  None,
  None,
  0,
  datetime.datetime(2023, 9, 9, 18, 28, 33, 121253),
  datetime.datetime(2023, 9, 9, 18, 28, 33, 121253),
  'llm',
  'processing',
  N

- Memory管理

In [27]:
from bida import ChatLLM
from bida import MessagePairBufferController

llm = ChatLLM(
    model_type="baidu",
    stream_callback=ChatLLM.stream_callback_func,
    buffer_controller=MessagePairBufferController(pair_count=1) # 每次只保留一轮对话
    ) 

result = llm.chat("你是一个服务助理，请简洁回答我的问题。我的名字是老周，年龄30岁。")


好的，老周。请问有什么我可以帮助您的吗？

In [28]:
result = llm.chat("我的名字是？")

您告诉我您的名字是老周。如果您有任何需要帮助的地方，请随时告诉我。

In [29]:
result = llm.chat("我多大了？")

您说过您是年轻人。如果您有其他疑问或者需要帮助，请随时告诉我。

In [31]:
# 可以看到大模型已经忘记年龄, 因为第一次对话已经归档，不再做为历史消息提交给大模型
llm.conversation.history_for_llm

[Message(id='aa0d2c08-892a-4eff-961f-825a90894ceb', role='user', content='我的名字是？', display_content='我的名字是？', name=None, function_call=None, tokens=50, start_time=datetime.datetime(2023, 9, 9, 18, 30, 16, 201055), end_time=datetime.datetime(2023, 9, 9, 18, 30, 16, 201055), category=<MessageCategory.llm: 'llm'>, status=<MessageStatus.completed: 'completed'>, link_message_id=None, update_time=datetime.datetime(2023, 9, 9, 18, 30, 18, 226128)),
 Message(id='24b40d94-dffc-4b0a-a6fc-b3d1e40594e8', role='assistant', content='您告诉我您的名字是老周。如果您有任何需要帮助的地方，请随时告诉我。', display_content='您告诉我您的名字是老周。如果您有任何需要帮助的地方，请随时告诉我。', name=None, function_call=None, tokens=30, start_time=datetime.datetime(2023, 9, 9, 18, 30, 18, 114776), end_time=datetime.datetime(2023, 9, 9, 18, 30, 18, 226128), category=<MessageCategory.llm: 'llm'>, status=<MessageStatus.completed: 'completed'>, link_message_id=None, update_time=datetime.datetime(2023, 9, 9, 18, 30, 18, 226128)),
 Message(id='4cbac63c-1a6d-432a-98d0-a222fb4dd4a3',